<a href="https://colab.research.google.com/github/ramonVDAKKER/teaching-data-science-emas/blob/main/notebooks/intro_pca.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PRINCIPAL COMPONENT ANALYSIS

The notebook contains illustrations and exercises corresponding to the topic <i>principal component analysis</i>.

## Import packages

In [ ]:
from sklearn.decomposition import PCA
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
%pylab inline
from matplotlib.patches import FancyArrowPatch
from sklearn.decomposition import PCA
from numpy.random import RandomState
from numpy import linalg as LA

# I. Illustration PCA

## PCA illustration 2-d

### Generate data $X_1,\dots,X_n$ from bivariate Gaussian distribution and generate scatter plot

In [ ]:
# Parameters
n = 200
mean = [1, 3]
rho = 0.75
sigma_1 = 1
sigma_2 = np.sqrt(1)
cov = rho * sigma_1 * sigma_2
covMatrix = [[sigma_1 ** 2, cov], [cov, sigma_2 ** 2]]
# Simulate data:
X = np.random.multivariate_normal(mean, covMatrix, n)
# Scatter plot:
plt.scatter(X[:, 0], X[:, 1])
plt.axis('equal')
plt.xlabel('$X_1$')
plt.ylabel('$X_2$')
plt.show()

### Determine principal components

Note: as long as $n\geq 2$ we can identify 2 principal components)

Estimate covariance matrix:

In [ ]:
hatSigma = np.cov(X, rowvar=False)
print(hatSigma)

Determine spectral decomposition using linear algebra package:

In [ ]:
eigenvalues, matrixEigenvectors = LA.eig(hatSigma)
print("Eigenvalues:")
print(eigenvalues)
print("Matrix with normalized eigenvectors (in columns):")
print(matrixEigenvectors)

Warning: it is not guaranteed that LA.eig yields <i>ordered</i> eigenvalues.

Check orthogonality eigenvectors and unit lengths:

In [ ]:
np.dot(matrixEigenvectors.T, matrixEigenvectors)

##### Question:
- Determine the principal components.
- Determine the % of variance explained by first component.

In [ ]:
100 * eigenvalues[1] / (eigenvalues[0] + eigenvalues[1])

The actual/true principal components:

In [ ]:
eigenvalues, matrixEigenvectors = LA.eig(covMatrix)
print("Eigenvalues:")
print(eigenvalues)
print("Matrix with normalized eigenvectors (in columns):")
print(matrixEigenvectors)

The <a href="http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html">pca module in the Scikit package</a> can do the work for us:  

In [ ]:
Xmean = X.mean(axis=0)
Xdemean = X - Xmean # recall that data needs to be de-meaned
n_princ_comp = 2
pca = PCA(n_components=n_princ_comp, svd_solver='full')
pca.fit(Xdemean)

In [ ]:
components = pca.components_
print("Principal components using SciKit package:")
print(components)
eigenvalues2 = np.power(pca.singular_values_, 2) / (len(X) - 1) # The SciKit package computes singular value of Xdemeaned
print("Eigenvalues using SciKit package:")
print(eigenvalues2)

Note that Scikit organizes principal components in rows!  (For $d=p$ confusion could arise. So always try $d<p$ as well.)

In [ ]:
vr = pca.explained_variance_ratio_  # \lambda_j / \sum_{j=1}^p \lambda_j  for j=1,...,d
y_pos = np.arange(len(vr))
plt.bar(y_pos, vr)
plt.bar(y_pos, 100 * vr, align='center', alpha=0.5)
plt.ylabel('%')
plt.xlabel('no. principal component')
plt.title('% of variance explained by x-th principal component')
plt.show()
print("The first %d principal components explain %d percent of the variance" % (len(vr), 100*sum(vr)))

### Plot the principal components using the corresponding eigenvalue (which equals $\operatorname{var}\left(  w_j^\prime X\right)$ ) as length of the vector

In [ ]:
def draw_vector(v0, v1, ax=None):
    ax = ax or plt.gca()
    arrowprops=dict(arrowstyle='->',
                    linewidth=2,color='b',
                    shrinkA=0, shrinkB=0)
    ax.annotate('', v1, v0, arrowprops=arrowprops)

# plot data
plt.scatter(X[:, 0], X[:, 1], alpha=0.2)

for length, vector in zip(pca.explained_variance_, pca.components_):
    v = vector * 2 * np.sqrt(length)
    draw_vector(Xmean, Xmean + v)
plt.axis('equal');

### Question:
Repeat the above for $\rho=0.95$ and $\rho=0$ and for $n=30$ and $n=10000$.

# II. Application PCA as dimension reduction/ data compression technique


<a href="https://www.cs.ucsb.edu/~mturk/Papers/jcn.pdf">Turk and Pentland (1991)</a> is (one of the) first paper(s) to use PCA in the context of face recognition. See <a href="http://www.face-rec.org/">link</a> for several datasets and (modern) algorithms.

### Retrieve  <a href="http://scikit-learn.org/stable/datasets/olivetti_faces.html">Olivetti faces dataset</a>

   

"The dataset contains a set of face images taken between April 1992 and April 1994 at AT&T Laboratories Cambridge. As described on the original website:

There are ten different images of each of 40 distinct subjects. For some subjects, the images were taken at different times, varying the lighting, facial expressions (open / closed eyes, smiling / not smiling) and facial details (glasses / no glasses). All the images were taken against a dark homogeneous background with the subjects in an upright, frontal position (with tolerance for some side movement).
The image is quantized to 256 grey levels and stored as unsigned 8-bit integers; the loader will convert these to floating point values on the interval [0, 1], which are easier to work with for many algorithms.

The “target” for this database is an integer from 0 to 39 indicating the identity of the person pictured; however, with only 10 examples per class, this relatively small dataset is more interesting from an unsupervised or semi-supervised perspective.

The original dataset consisted of 92 x 112, while the version available here consists of 64x64 images."

In [ ]:
from sklearn.datasets import fetch_olivetti_faces # dataset
dataset = fetch_olivetti_faces(shuffle=True, random_state=RandomState()) # retrieve data
faces = dataset.data # array containing data, columns=features correspond to the 64 x 64 pixels
n_samples, n_features = faces.shape
print("The dataset consists of %d pictures" % n_samples)
print("Each picture contains %d features which corresponds to 64x64 (pixels)" % n_features)

### Plot a few faces

In [ ]:
# function to plot faces
def plot_gallery(title, images, image_shape, n_col, n_row):
    plt.figure(figsize=(3. * n_col, 3.26 * n_row))
    plt.suptitle(title, size=16)
    for i, comp in enumerate(images):
        plt.subplot(n_row, n_col, i + 1)
        vmax = max(comp.max(), -comp.min())
        plt.imshow(comp.reshape(image_shape), cmap=plt.cm.gray,
                   interpolation='nearest',
                   vmin=-vmax, vmax=vmax)
        plt.xticks(())
        plt.yticks(())
    plt.subplots_adjust(0.01, 0.05, 0.99, 0.93, 0.04, 0.)
plot_gallery("True faces", faces[10:15,:], (64,64), 5,1)
plt.show()

### Center data to prepare for PCA

In [ ]:
averageFace = faces.mean(axis=0)
faces_centered = faces - averageFace
plot_gallery("Average face", averageFace.reshape(n_features,1).T,(64,64),5,1)
plot_gallery("True faces", faces[10:15,:],(64,64),5,1)
plot_gallery("Centered faces", faces_centered[10:15,:],(64,64),5,1)
plt.show()


### Compute PCA and check % variance explained

In [ ]:
n_princ_comp = 50  # here you can adapt the no. of principal components [max=400; why?],  please use >=5!!!
#
pca = PCA(n_components=n_princ_comp, svd_solver='full')
pca.fit(faces_centered)
vr = pca.explained_variance_ratio_  # \lambda_j / \sum_{j=1}^p \lambda_j  for j=1,...,d
y_pos = np.arange(len(vr))
plt.bar(y_pos, vr)
plt.bar(y_pos, 100*vr, align="center", alpha=0.5)
plt.ylabel("%")
plt.xlabel("no. principal component")
plt.title("% of variance explained by x-th principal component")
plt.show()
print("The first %d principal components explain %d percent of the variance" % (len(vr), 100*sum(vr)))


In [ ]:
components = pca.components_
sv = pca.singular_values_
print("The average face:")
plot_gallery("Average face", averageFace.reshape(n_features,1).T, (64, 64), 10, 1)
plt.show()
print("The 5 faces, called eigen faces, corresponding to the first 10 principal components:")
plot_gallery("Eigen faces", components[0:10, :],(64, 64), 10, 1)
plt.show()

### Let us construct a new face by taking linear combination of eigen faces (weights=1) and adding average face:

In [ ]:
newFace =  np.sum(components[0:10,:], axis=0)
plot_gallery("New face", newFace.reshape(n_features,1).T, (64, 64), 10, 1)
plt.show()

### Reconstruct faces in dataset [on basis of average face and the eigen faces]

In [ ]:
scores = pca.transform(faces_centered)
reconstructedFaces = pca.inverse_transform(scores) + averageFace
plot_gallery("True faces", faces[10:15,:], (64, 64), 10, 1)
plot_gallery("Reconstructed Faces based on PCA using d=" + str(n_princ_comp), reconstructedFaces[10:15,:], (64, 64), 10, 1)
plt.show()
